In [1]:
import fasttext
import pandas as pd
import re

In [8]:
filepath = 'data.xlsx'

data = pd.read_excel(filepath)

In [9]:
data.head(5)

,Numéro d'application,Date d'application,Numero de publication,date de publication,CPC,IPC,claim,description
0,17167919,2017-04-25,3264223,2018-01-03,"['B23K26-361', 'H04M1-0266', 'B23K26-402', 'G0...","['C08J7-04', 'H04M1-02', 'G06F1-16', 'B23K26-3...","<!-- EPO <DP n=""18""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><p id=""p0001"" num=""0001..."
1,17166775,2017-04-18,3235458,2017-10-25,"['A61B17-68', 'A61B17-888', 'A61B17-8685', 'A6...","['A61B17-88', 'A61B17-68', 'A61B17-86']","<!-- EPO <DP n=""20""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">FIE..."
2,17166069,2017-04-11,3249094,2017-11-29,"['D06F58-206', 'D06F58-20']","['D06F58-20', 'D06F58-02']","<!-- EPO <DP n=""22""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001""><b>..."
3,18771363,2018-03-05,3605724,2020-02-05,"['H01P1-2138', 'H01P5-107', 'H01P1-2088', 'H01...","['H01P3-12', 'H01P1-213', 'H01P5-107']","<!-- EPO <DP n=""62""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">Tec..."
4,19881950,2019-11-01,3713245,2020-09-23,"['H04N21-4345', 'H04N21-4821', 'H04N21-4316']","['H04N21-482', 'H04N21-262']","<!-- EPO <DP n=""14""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><p id=""p0001"" num=""0001..."


In [10]:
import re
import ast
from collections import Counter

def split_code(l):

    # Cette fonction prend une liste de codes alphanumériques et retourne une nouvelle liste
    # contenant les sections les plus fréquentes à chaque position, tout en respectant
    # la validité des sélections précédentes pour maintenir la correspondance avec les codes originaux.
    
    l = ast.literal_eval(l)
    splitted_code_list = []
    splitted_code = []

    # Diviser chaque code et séparer les lettres et les chiffres
    for elt in l:
        elt = elt.split('-')
        letters = list(filter(None, re.split(r'[0-9]', elt[0])))
        digits = list(filter(None, re.split(r'[A-Z]', elt[0])))
        code = [sub[item] for item in range(len(digits)) for sub in [letters, digits]]
        code.append(elt[1])
        splitted_code.append(code)

    # Déterminer les sections les plus prédominantes tout en maintenant la validité
    result = []
    filtered_codes = splitted_code

    for i in range(len(splitted_code[0])):
        # Extraire les éléments ièmes des codes filtrés
        ith_elements = [code[i] for code in filtered_codes]
        
        # Trouver l'élément le plus commun
        most_common, _ = Counter(ith_elements).most_common(1)[0]
        
        # Ajouter l'élément le plus commun au résultat
        result.append(most_common)
        
        # Filtrer les codes pour la prochaine itération
        filtered_codes = [code for code in filtered_codes if code[i] == most_common]

    # We add a the concatenated code to the result
    #result.append('-'.join(result[0:3]) + '-' + result[3])

    return result

# Exemple d'utilisation
res = split_code("['B23K26-361', 'H04C1-0266', 'B23K26-402', 'M04C1-0256', 'U04C1-0246']")
print(res)

['B', '23', 'K', '26', '361']


In [11]:
cpc_cleaned = data['CPC'].apply(split_code).apply(pd.Series)
cpc_cleaned.columns = ['CPC_section','CPC_class','CPC_subclass','CPC_group','CPC_subgroup']

# We add the columns to the original dataframe
data = pd.concat([data, cpc_cleaned], axis=1)

data.drop(['Numéro d\'application', 'Date d\'application','Numero de publication', 'date de publication'], axis=1, inplace=True)

data.head()

,CPC,IPC,claim,description,CPC_section,CPC_class,CPC_subclass,CPC_group,CPC_subgroup
0,"['B23K26-361', 'H04M1-0266', 'B23K26-402', 'G0...","['C08J7-04', 'H04M1-02', 'G06F1-16', 'B23K26-3...","<!-- EPO <DP n=""18""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><p id=""p0001"" num=""0001...",B,23,K,26,361
1,"['A61B17-68', 'A61B17-888', 'A61B17-8685', 'A6...","['A61B17-88', 'A61B17-68', 'A61B17-86']","<!-- EPO <DP n=""20""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">FIE...",A,61,B,17,68
2,"['D06F58-206', 'D06F58-20']","['D06F58-20', 'D06F58-02']","<!-- EPO <DP n=""22""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001""><b>...",D,06,F,58,206
3,"['H01P1-2138', 'H01P5-107', 'H01P1-2088', 'H01...","['H01P3-12', 'H01P1-213', 'H01P5-107']","<!-- EPO <DP n=""62""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">Tec...",H,01,P,1,2138
4,"['H04N21-4345', 'H04N21-4821', 'H04N21-4316']","['H04N21-482', 'H04N21-262']","<!-- EPO <DP n=""14""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><p id=""p0001"" num=""0001...",H,04,N,21,4345


In [13]:
import re
import nltk
import string
from nltk.corpus import stopwords

nltk.download('stopwords')

def clean_text(text):
    # Remove HTML tags
    text = re.sub('<.*?>', ' ', text)
    
    # Remove specific patterns (e.g., '-->')
    text = re.sub('-->', '', text)
    
    # Remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove numbers
    text = re.sub(r'[0-9]', '', text)
    
    # Remove extra whitespace
    text = " ".join(text.split())
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    text = ' '.join(filtered_words)
    
    return text


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kevmu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
data['description'] = data['description'] + ' ' + data['claim']
data['description'] = data['description'].apply(clean_text)

data.drop(['claim'], axis=1, inplace=True)

In [15]:
data.head(1)

,CPC,IPC,description,CPC_section,CPC_class,CPC_subclass,CPC_group,CPC_subgroup
0,"['B23K26-361', 'H04M1-0266', 'B23K26-402', 'G0...","['C08J7-04', 'H04M1-02', 'G06F1-16', 'B23K26-3...",present invention relates cover window display...,B,23,K,26,361


In [16]:
import os
import pandas as pd

def generate_fasttext_files(data):
    """
    Main function to generate FastText format files for hierarchical classification.
    """
    # Create a main directory to store the FastText files
    output_dir = "fasttext_files"
    os.makedirs(output_dir, exist_ok=True)

    # Step 1: Generate FastText file for CPC section
    section_dir = os.path.join(output_dir, "sections")
    os.makedirs(section_dir, exist_ok=True)
    create_fasttext_file(data, "CPC_section", os.path.join(section_dir, "fasttext_section.txt"))

    # Get unique CPC sections
    unique_sections = data["CPC_section"].unique()

    # Step 2: For each section, generate FastText file for CPC class
    for section in unique_sections:
        filtered_data = filter_data(data, "CPC_section", section)
        class_dir = os.path.join(section_dir, f"section_{section}")
        os.makedirs(class_dir, exist_ok=True)
        output_file = os.path.join(class_dir, f"fasttext_section_{section}_class.txt")
        create_fasttext_file(filtered_data, "CPC_class", output_file)

        # Get unique CPC classes within this section
        unique_classes = filtered_data["CPC_class"].unique()

        # Step 3: For each class, generate FastText file for CPC subclass
        for cpc_class in unique_classes:
            filtered_data_class = filter_data(filtered_data, "CPC_class", cpc_class)
            subclass_dir = os.path.join(class_dir, f"class_{cpc_class}")
            os.makedirs(subclass_dir, exist_ok=True)
            output_file = os.path.join(subclass_dir, f"fasttext_section_{section}_class_{cpc_class}_subclass.txt")
            create_fasttext_file(filtered_data_class, "CPC_subclass", output_file)

            # Get unique CPC subclasses within this class
            unique_subclasses = filtered_data_class["CPC_subclass"].unique()

            # Step 4: For each subclass, generate FastText file for CPC group
            for subclass in unique_subclasses:
                filtered_data_subclass = filter_data(filtered_data_class, "CPC_subclass", subclass)
                group_dir = os.path.join(subclass_dir, f"subclass_{subclass}")
                os.makedirs(group_dir, exist_ok=True)
                output_file = os.path.join(group_dir, f"fasttext_section_{section}_class_{cpc_class}_subclass_{subclass}_group.txt")
                create_fasttext_file(filtered_data_subclass, "CPC_group", output_file)

                # Get unique CPC groups within this subclass
                unique_groups = filtered_data_subclass["CPC_group"].unique()

                # Step 5: For each group, generate FastText file for CPC subgroup
                for group in unique_groups:
                    filtered_data_group = filter_data(filtered_data_subclass, "CPC_group", group)
                    subgroup_dir = os.path.join(group_dir, f"group_{group}")
                    os.makedirs(subgroup_dir, exist_ok=True)
                    output_file = os.path.join(subgroup_dir, f"fasttext_section_{section}_class_{cpc_class}_subclass_{subclass}_group_{group}_subgroup.txt")
                    create_fasttext_file(filtered_data_group, "CPC_subgroup", output_file)

def create_fasttext_file(data, target_column, output_file):
    """
    Creates a FastText formatted file for the given target column and saves it to the specified output file.
    """
    with open(output_file, 'w') as file:
        for index, row in data.iterrows():
            text = row['description']
            label = row[target_column]
            file.write(f"__label__{label} {text}\n")

def filter_data(data, filter_column, filter_value):
    """
    Filters the dataframe based on the specified column and value.
    """
    return data[data[filter_column] == filter_value]




In [23]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.3)

In [49]:
train_data.head()

,CPC,IPC,description,CPC_section,CPC_class,CPC_subclass,CPC_group,CPC_subgroup
2612,"['B21D43-00', 'B21D45-00', 'B26D7-32', 'B21D43...","['B21D43-00', 'B21D45-00', 'B23Q7-00', 'B23K37...",present invention relates sheet material machi...,B,21,D,43,00
48733,"['H05K13-086', 'H05K13-021']","['H05K13-00', 'H05K13-02', 'H05K13-08']",technical field present description relates ex...,H,05,K,13,086
18858,['H01Q3-06'],['H01Q3-06'],background field disclosure relates base stati...,H,01,Q,3,06
27070,"['F16K31-406', 'F04B27-1804', 'F04B2027-1845',...","['F04B39-08', 'F16K31-06', 'F16K31-363', 'F04B...",technical field present invention relates capa...,F,04,B,2027,1845
25623,"['A61G5-04', 'B60W30-14', 'B60Q9-008', 'G08G1-...","['A61G5-04', 'G08G1-16', 'G05D1-02', 'B60T7-00...",technical field present invention relates cont...,B,60,W,2050,143


In [24]:

generate_fasttext_files(train_data)  # Call the function to generate FastText files


In [25]:
import os
import fasttext

def train_fasttext_models(base_dir):
    """
    Train FastText models for each hierarchical level.
    """
    # Step 1: Train model for CPC section
    section_file = os.path.join(base_dir, "sections", "fasttext_section.txt")
    section_model = fasttext.train_supervised(input=section_file)
    section_model.save_model(os.path.join("fasttext_files/sections", "section_model.bin"))

    # Get subfolders for sections
    section_dirs = [d for d in os.listdir(os.path.join(base_dir, "sections")) if os.path.isdir(os.path.join(base_dir, "sections", d))]

    # Step 2: Train model for each CPC class
    for section_dir in section_dirs:
        class_file = os.path.join(base_dir, "sections", section_dir, f"fasttext_{section_dir}_class.txt")
        class_model = fasttext.train_supervised(input=class_file)
        class_model.save_model(os.path.join(base_dir, "sections", section_dir, "class_model.bin"))

        # Get subfolders for classes
        class_dirs = [d for d in os.listdir(os.path.join(base_dir, "sections", section_dir)) if os.path.isdir(os.path.join(base_dir, "sections", section_dir, d))]

        # Step 3: Train model for each CPC subclass
        for class_dir in class_dirs:
            subclass_file = os.path.join(base_dir, "sections", section_dir, class_dir, f"fasttext_{section_dir}_{class_dir}_subclass.txt")
            subclass_model = fasttext.train_supervised(input=subclass_file)
            subclass_model.save_model(os.path.join(base_dir, "sections", section_dir, class_dir, "subclass_model.bin"))

            # Get subfolders for subclasses
            subclass_dirs = [d for d in os.listdir(os.path.join(base_dir, "sections", section_dir, class_dir)) if os.path.isdir(os.path.join(base_dir, "sections", section_dir, class_dir, d))]

            # Step 4: Train model for each CPC group
            for subclass_dir in subclass_dirs:
                group_file = os.path.join(base_dir, "sections", section_dir, class_dir, subclass_dir, f"fasttext_{section_dir}_{class_dir}_{subclass_dir}_group.txt")
                group_model = fasttext.train_supervised(input=group_file)
                group_model.save_model(os.path.join(base_dir, "sections", section_dir, class_dir, subclass_dir, "group_model.bin"))

                # Get subfolders for groups
                group_dirs = [d for d in os.listdir(os.path.join(base_dir, "sections", section_dir, class_dir, subclass_dir)) if os.path.isdir(os.path.join(base_dir, "sections", section_dir, class_dir, subclass_dir, d))]

                # Step 5: Train model for each CPC subgroup
                for group_dir in group_dirs:
                    subgroup_file = os.path.join(base_dir, "sections", section_dir, class_dir, subclass_dir, group_dir, f"fasttext_{section_dir}_{class_dir}_{subclass_dir}_{group_dir}_subgroup.txt")
                    subgroup_model = fasttext.train_supervised(input=subgroup_file)
                    subgroup_model.save_model(os.path.join(base_dir, "sections", section_dir, class_dir, subclass_dir, group_dir, "subgroup_model.bin"))

# Base directory where the FastText files are stored
base_dir = "fasttext_files"


In [26]:

# Train the FastText models
model = train_fasttext_models(base_dir)



In [69]:
train_data['description'][1]


'field invention present disclosure relates implantable screws particularly disclosure relates implantable screws configured provide compression upon bone screw implanted background invention broken bone must carefully stabilized supported strong enough handle bodys weight movement last century physicians relied casts splints support stabilize bone outside body advent sterile surgical procedures reduced risk infection allowing doctors internally set stabilize fractured bones surgical procedure set fracture bone fragments first repositioned reduced normal alignment held together special implants plates screws nails wires screws used internal fixation often type implant although screw simple device different designs based type fracture screw used screws come different sizes use bones different sizes screws used alone hold fracture well plates rods nails bone heals screws may either left place removed many instances desired inserted screw provide compression bone joint fracture line reduc

In [37]:
train_data[['CPC_section','CPC_class','CPC_subclass','CPC_group','CPC_subgroup']]

,CPC_section,CPC_class,CPC_subclass,CPC_group,CPC_subgroup
2612,B,21,D,43,00
48733,H,05,K,13,086
18858,H,01,Q,3,06
27070,F,04,B,2027,1845
25623,B,60,W,2050,143
...,...,...,...,...,...
40886,B,01,J,20,28004
29879,G,09,G,5,003
40323,C,09,G,1,02
21153,G,06,Q,10,06


In [39]:
cpc_codes = {
        "A": "Human necessities",
        "B": "Performing operations; transporting",
        "C": "Chemistry; metallurgy",
        "D": "Textiles; paper",
        "E": "Fixed constructions",
        "F": "Mechanical engineering; lighting; heating; weapons; blasting engines or pumps",
        "G": "Physics",
        "H": "Electricity",
        "Y": "General tagging of new technological developments; general tagging of cross-sectional technologies spanning over several sections of the IPC; technical subjects covered by former USPC cross-reference art collections [XRACs] and digests"
    }

In [40]:
def get_cpc_description(cpc_section):
    return cpc_codes.get(cpc_section, "Unknown CPC section")

def classify_text_hierarchically(text):
    # Function to classify text at each hierarchical level

    def load_and_predict(model_path, text):
        if not os.path.exists(model_path):
            return None, None
        model = fasttext.load_model(model_path)
        predicted_class, probability = model.predict(text)
        return predicted_class[0].split("__label__")[-1], probability[0]

    result = {}
    
    # Level 1: CPC Section
    section_model_path = os.path.join(base_dir, "section_model.bin")
    cpc_section, section_prob = load_and_predict(section_model_path, text)
    if cpc_section is None:
        result['section'] = "Model not found"
        return result

    result['section'] = {
        'code': cpc_section,
        'description': get_cpc_description(cpc_section),
        'probability': section_prob
    }
    
    # Level 2: CPC Class
    class_model_path = os.path.join(base_dir, f"section_{cpc_section}", "class_model.bin")
    cpc_class, class_prob = load_and_predict(class_model_path, text)
    if cpc_class is None:
        result['class'] = "Model not found"
        return result

    result['class'] = {'code': cpc_class, 'probability': class_prob}

    # Level 3: CPC Subclass
    subclass_model_path = os.path.join(base_dir, f"section_{cpc_section}", f"class_{cpc_class}", "subclass_model.bin")
    cpc_subclass, subclass_prob = load_and_predict(subclass_model_path, text)
    if cpc_subclass is None:
        result['subclass'] = "Model not found"
        return result

    result['subclass'] = {'code': cpc_subclass, 'probability': subclass_prob}

    # Level 4: CPC Group
    group_model_path = os.path.join(base_dir, f"section_{cpc_section}", f"class_{cpc_class}", f"subclass_{cpc_subclass}", "group_model.bin")
    cpc_group, group_prob = load_and_predict(group_model_path, text)
    if cpc_group is None:
        result['group'] = "Model not found"
        return result

    result['group'] = {'code': cpc_group, 'probability': group_prob}

    # Level 5: CPC Subgroup
    subgroup_model_path = os.path.join(base_dir, f"section_{cpc_section}", f"class_{cpc_class}", f"subclass_{cpc_subclass}", f"group_{cpc_group}", "subgroup_model.bin")
    cpc_subgroup, subgroup_prob = load_and_predict(subgroup_model_path, text)
    if cpc_subgroup is None:
        result['subgroup'] = "Model not found"
        return result

    result['subgroup'] = {'code': cpc_subgroup, 'probability': subgroup_prob}

    # Combine the full CPC code
    result['full_cpc_code'] = f"{cpc_section}{cpc_class}{cpc_subclass}{cpc_group}/{cpc_subgroup}"

    return result

In [41]:
def format_classification_result(result):
    if 'section' in result and isinstance(result['section'], dict):
        section_desc = f"{result['section']['code']} - {result['section']['description']} (Probability: {result['section']['probability']:.2f})"
    else:
        section_desc = "Section model not found"
    
    cpc_class = result.get('class', "Class model not found")
    if isinstance(cpc_class, dict):
        class_desc = f"{cpc_class['code']} (Probability: {cpc_class['probability']:.2f})"
    else:
        class_desc = cpc_class
    
    cpc_subclass = result.get('subclass', "Subclass model not found")
    if isinstance(cpc_subclass, dict):
        subclass_desc = f"{cpc_subclass['code']} (Probability: {cpc_subclass['probability']:.2f})"
    else:
        subclass_desc = cpc_subclass
    
    cpc_group = result.get('group', "Group model not found")
    if isinstance(cpc_group, dict):
        group_desc = f"{cpc_group['code']} (Probability: {cpc_group['probability']:.2f})"
    else:
        group_desc = cpc_group
    
    cpc_subgroup = result.get('subgroup', "Subgroup model not found")
    if isinstance(cpc_subgroup, dict):
        subgroup_desc = f"{cpc_subgroup['code']} (Probability: {cpc_subgroup['probability']:.2f})"
    else:
        subgroup_desc = cpc_subgroup
    
    full_cpc_code = result.get('full_cpc_code', "Full CPC code not found")

    formatted_result = {
        'Section' : section_desc,
        'Class' : class_desc,
        'Subclass' : subclass_desc,
        'Subgroup' : subgroup_desc,
        'Full CPC Code' : full_cpc_code
    }

    return formatted_result

In [48]:
import os
import fasttext
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from concurrent.futures import ThreadPoolExecutor

def load_models(base_dir):
    """
    Load all FastText models into a dictionary for faster access during evaluation.
    
    Args:
    base_dir (str): Base directory where the models are stored.
    
    Returns:
    dict: Dictionary of loaded models.
    """
    models = {}
    
    # Load section model
    section_model_path = os.path.join(base_dir, "sections", "section_model.bin")
    if os.path.exists(section_model_path):
        models['section'] = fasttext.load_model(section_model_path)
    
    # Load class models
    section_dirs = [d for d in os.listdir(os.path.join(base_dir, "sections")) if os.path.isdir(os.path.join(base_dir, "sections", d))]
    for section_dir in section_dirs:
        class_model_path = os.path.join(base_dir, "sections", section_dir, "class_model.bin")
        if os.path.exists(class_model_path):
            models[f'class_{section_dir}'] = fasttext.load_model(class_model_path)
        
        # Load subclass models
        class_dirs = [d for d in os.listdir(os.path.join(base_dir, "sections", section_dir)) if os.path.isdir(os.path.join(base_dir, "sections", section_dir, d))]
        for class_dir in class_dirs:
            subclass_model_path = os.path.join(base_dir, "sections", section_dir, class_dir, "subclass_model.bin")
            if os.path.exists(subclass_model_path):
                models[f'subclass_{section_dir}_{class_dir}'] = fasttext.load_model(subclass_model_path)
            
            # Load group models
            subclass_dirs = [d for d in os.listdir(os.path.join(base_dir, "sections", section_dir, class_dir)) if os.path.isdir(os.path.join(base_dir, "sections", section_dir, class_dir, d))]
            for subclass_dir in subclass_dirs:
                group_model_path = os.path.join(base_dir, "sections", section_dir, class_dir, subclass_dir, "group_model.bin")
                if os.path.exists(group_model_path):
                    models[f'group_{section_dir}_{class_dir}_{subclass_dir}'] = fasttext.load_model(group_model_path)
                
                # Load subgroup models
                group_dirs = [d for d in os.listdir(os.path.join(base_dir, "sections", section_dir, class_dir, subclass_dir)) if os.path.isdir(os.path.join(base_dir, "sections", section_dir, class_dir, subclass_dir, d))]
                for group_dir in group_dirs:
                    subgroup_model_path = os.path.join(base_dir, "sections", section_dir, class_dir, subclass_dir, group_dir, "subgroup_model.bin")
                    if os.path.exists(subgroup_model_path):
                        models[f'subgroup_{section_dir}_{class_dir}_{subclass_dir}_{group_dir}'] = fasttext.load_model(subgroup_model_path)
    
    return models

def load_and_predict(model, text):
    """
    Predict the class and probability for a given text using a loaded FastText model.
    
    Args:
    model (fasttext.FastText._FastText): Loaded FastText model.
    text (str): Text to classify.
    
    Returns:
    tuple: Predicted class and probability.
    """
    predicted_class, probability = model.predict(text)
    return predicted_class[0].split("__label__")[-1], probability[0]

def evaluate_single_text(row, models):
    """
    Evaluate a single text by predicting its hierarchical CPC codes.
    
    Args:
    row (pd.Series): A row from the test DataFrame containing the description and true labels.
    models (dict): Dictionary of loaded models.
    
    Returns:
    dict: Dictionary containing the true and predicted labels for the text at each level.
    """
    text = row['description']
    result = {
        'section': {'true': row['CPC_section'], 'pred': 'Unknown'},
        'class': {'true': row['CPC_class'], 'pred': 'Unknown'},
        'subclass': {'true': row['CPC_subclass'], 'pred': 'Unknown'},
        'group': {'true': row['CPC_group'], 'pred': 'Unknown'},
        'subgroup': {'true': row['CPC_subgroup'], 'pred': 'Unknown'}
    }
    
    # Level 1: CPC Section
    section_model = models.get('section')
    if section_model:
        cpc_section, _ = load_and_predict(section_model, text)
        result['section']['pred'] = cpc_section

    if result['section']['true'] != result['section']['pred']:
        return result

    # Level 2: CPC Class
    class_model = models.get(f'class_section_{result["section"]["pred"]}')
    if class_model:
        cpc_class, _ = load_and_predict(class_model, text)
        result['class']['pred'] = cpc_class

    if result['class']['true'] != result['class']['pred']:
        return result

    # Level 3: CPC Subclass
    subclass_model = models.get(f'subclass_section_{result["section"]["pred"]}_class_{result["class"]["pred"]}')
    if subclass_model:
        cpc_subclass, _ = load_and_predict(subclass_model, text)
        result['subclass']['pred'] = cpc_subclass

    if result['subclass']['true'] != result['subclass']['pred']:
        return result

    # Level 4: CPC Group
    group_model = models.get(f'group_section_{result["section"]["pred"]}_class_{result["class"]["pred"]}_subclass_{result["subclass"]["pred"]}')
    if group_model:
        cpc_group, _ = load_and_predict(group_model, text)
        result['group']['pred'] = cpc_group

    if result['group']['true'] != result['group']['pred']:
        return result

    # Level 5: CPC Subgroup
    subgroup_model = models.get(f'subgroup_section_{result["section"]["pred"]}_class_{result["class"]["pred"]}_subclass_{result["subclass"]["pred"]}_group_{result["group"]["pred"]}')
    if subgroup_model:
        cpc_subgroup, _ = load_and_predict(subgroup_model, text)
        result['subgroup']['pred'] = cpc_subgroup

    return result

def evaluate(test_data, base_dir):
    """
    Evaluate the hierarchical classification models on the given test DataFrame.
    
    Args:
    test_data (pd.DataFrame): DataFrame containing the test texts and their true labels.
    base_dir (str): Base directory where the models are stored.
    
    Returns:
    dict: Dictionary containing accuracy and classification reports for each level.
    """
    models = load_models(base_dir)
    
    results = {
        'section': {'true': [], 'pred': []},
        'class': {'true': [], 'pred': []},
        'subclass': {'true': [], 'pred': []},
        'group': {'true': [], 'pred': []},
        'subgroup': {'true': [], 'pred': []}
    }
    
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(evaluate_single_text, row, models) for index, row in test_data.iterrows()]
        
        for future in futures:
            result = future.result()
            for level in results.keys():
                results[level]['true'].append(result[level]['true'])
                results[level]['pred'].append(result[level]['pred'])
    
    # Calculate accuracy and classification report for each level
    evaluation_results = {}
    for level in results.keys():
        accuracy = accuracy_score(results[level]['true'], results[level]['pred'])
        report = classification_report(results[level]['true'], results[level]['pred'])
        evaluation_results[level] = {'accuracy': accuracy, 'report': report}
    
    return evaluation_results

# Example usage
# Assuming test_data is a DataFrame that you have already created from your train-test split
evaluation_results = evaluate(test_data, base_dir)
for level, metrics in evaluation_results.items():
    print(f"Level: {level}")
    print(f"Accuracy: {metrics['accuracy']:.2f}")
    print(f"Classification Report:\n{metrics['report']}")


c:\Users\kevmu\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\kevmu\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\kevmu\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\kevmu\miniconda3\Lib\site-pa

Level: section
Accuracy: 0.73
Classification Report:
              precision    recall  f1-score   support

           A       0.72      0.70      0.71      1951
           B       0.65      0.68      0.66      2452
           C       0.71      0.78      0.74      1524
           D       0.82      0.15      0.26       178
           E       0.33      0.00      0.01       272
           F       0.69      0.74      0.71      1409
           G       0.73      0.76      0.75      3258
           H       0.82      0.82      0.82      3898
           Y       0.00      0.00      0.00        58

    accuracy                           0.73     15000
   macro avg       0.61      0.51      0.52     15000
weighted avg       0.72      0.73      0.72     15000

Level: class
Accuracy: 0.43
Classification Report:
              precision    recall  f1-score   support

          01       0.71      0.50      0.59      2545
          02       0.67      0.41      0.50       953
          03       0.78     